# importig Training file 

In [37]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import category_encoders as ce
from sklearn import svm

from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

filename = "../input/tcdml1920-income-ind/tcd ml 2019-20 income prediction training (with labels).csv"
dataset = pd.read_csv(filename,delimiter = ',')
#checking what data is present
dataset.info()


/kaggle/input/tcdml1920-income-ind/tcd ml 2019-20 income prediction submission file example (random solutions).csv
/kaggle/input/tcdml1920-income-ind/tcd ml 2019-20 income prediction submission file.csv
/kaggle/input/tcdml1920-income-ind/tcd ml 2019-20 income prediction test (without labels).csv
/kaggle/input/tcdml1920-income-ind/tcd ml 2019-20 income prediction training (with labels).csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111993 entries, 0 to 111992
Data columns (total 12 columns):
Instance             111993 non-null int64
Year of Record       111552 non-null float64
Gender               104561 non-null object
Age                  111499 non-null float64
Country              111993 non-null object
Size of City         111993 non-null int64
Profession           111671 non-null object
University Degree    104623 non-null object
Wears Glasses        111993 non-null int64
Hair Color           104751 non-null object
Body Height [cm]     111993 non-null int64
Income in EUR  

# checking what data is present in 'wear glasses column' from given dataset

In [38]:

dataset['Wears Glasses']
dataset['Wears Glasses'].unique()
#checking how many null values are there in given dataset
dataset.isnull().sum()

Instance                0
Year of Record        441
Gender               7432
Age                   494
Country                 0
Size of City            0
Profession            322
University Degree    7370
Wears Glasses           0
Hair Color           7242
Body Height [cm]        0
Income in EUR           0
dtype: int64

# Applying preprocessing 

In [39]:

#Replacing 'NA' values in 'Hair Color' column with previous value
dataset["Hair Color"].fillna( method ='ffill', inplace = True)
dataset['Hair Color']=dataset['Hair Color'].replace('0', 'Unknown')
#dataset['Hair Color'] = dataset['Hair Color'].apply({'Blond':4, 'Black':1, 'Brown':2, 'Red':3, 'Unknown':0}.get)

#Replacing 'NA' values in 'Age' column with mean value
dataset['Age'] = dataset['Age'].fillna((dataset['Age'].mean()))

#Replacing 'NA' values in 'Year of Record' column with mean value
dataset['Year of Record']=dataset["Year of Record"].fillna(dataset['Year of Record'].mean())
#dataset['Year of Record'].fillna( method ='ffill', inplace = True)

#Replacing '0' values in 'University Degree' column with 'No'
dataset["University Degree"] = dataset["University Degree"].replace("0", "No")

#Replacing '0' values in 'University Degree' column with 'unknown'
dataset["Gender"] = dataset["Gender"].replace("0", "unknown")

# checking null values in dataset after replacing 

In [40]:

dataset.isnull().sum()

Instance                0
Year of Record          0
Gender               7432
Age                     0
Country                 0
Size of City            0
Profession            322
University Degree    7370
Wears Glasses           0
Hair Color              0
Body Height [cm]        0
Income in EUR           0
dtype: int64

# importing sklearn preprocessing and pipeline

In [41]:

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

#taking numeric value columns to apply normalisiation
num_colums = ['Age', 'Year of Record', 'Size of City', 'Body Height [cm]']
num_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())])
#taking catogerical columns and converting them to numerical using sklearn onehot encoding
catego_columns = ['Gender', 'University Degree', 'Country', 'Profession', 'Hair Color']
catego_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])


# Applying the transformation using sklearn columnTransformer

In [42]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_colums),
        ('cat', catego_transformer, catego_columns)])

# applying linear Regression in pipeline

In [43]:
from sklearn.ensemble import RandomForestClassifier
#copied given data into another variable
q = dataset

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LinearRegression())])
#taking the columns for testing and training
X = q[q.columns.difference(['Income in EUR'])]
y = q["Income in EUR"]
#checking null values in given data
q.isnull().sum()

Instance                0
Year of Record          0
Gender               7432
Age                     0
Country                 0
Size of City            0
Profession            322
University Degree    7370
Wears Glasses           0
Hair Color              0
Body Height [cm]        0
Income in EUR           0
dtype: int64

In [44]:
q

,Instance,Year of Record,Gender,Age,Country,Size of City,Profession,University Degree,Wears Glasses,Hair Color,Body Height [cm],Income in EUR
0,1,1997.0,unknown,41.0,Belarus,1239930,steel workers,Bachelor,0,Blond,193,61031.94416
1,2,1996.0,other,41.0,Singapore,1603504,safe event coordinator,Master,0,Black,186,91001.32764
2,3,2018.0,other,28.0,Norway,1298017,receivables/payables analyst,PhD,1,Brown,170,157982.17670
3,4,2006.0,other,33.0,Cuba,751903,fleet assistant,No,1,Black,171,45993.75793
4,5,2010.0,female,46.0,United Arab Emirates,95389,lead trainer,No,0,Blond,188,38022.16217
...,...,...,...,...,...,...,...,...,...,...,...,...
111988,111989,1980.0,female,29.0,Honduras,205274,industrial program compliance analyst,No,0,Brown,135,17185.12970
111989,111990,2006.0,female,36.0,Tunisia,1012826,senior policy advisor,Master,1,Brown,147,118806.53920
111990,111991,1993.0,female,36.0,Sri Lanka,60848,messenger,Master,1,Brown,153,289951.32940
111991,111992,2019.0,other,54.0,Denmark,848640,senior case support associate,No,0,Black,190,100046.52780


# splitiing the training data in 70/30% for training and testing

In [45]:


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
#testsize=500
#X_train =X[:-testsize]
#y_train =y[:-testsize]

#X_test=X[-testsize:]
#y_test=y[-testsize:]

# Applying the model to the training dataset

In [46]:

clf.fit(X_train,y_train)
#applying the model to test data and checking the score
print(clf.score(X_test,y_test))

0.7025133600543454


# Importing the test data file

In [47]:

testfilename = "../input/tcdml1920-income-ind/tcd ml 2019-20 income prediction test (without labels).csv"
testdata = pd.read_csv(testfilename,delimiter = ',')
testdata.isnull().sum()
testdata['University Degree'].unique()

array(['Master', 'Bachelor', 'No', nan, 'PhD', '0'], dtype=object)

# Applying the preprocessing to remove the null values in test data

In [48]:

#Replacing 'NA' values in 'Hair Color' column with previous value
testdata["Hair Color"].fillna( method ='ffill', inplace = True)
testdata['Hair Color']=testdata['Hair Color'].replace('0', 'Unknown')

#Replacing 'NA' values in 'Age' column with mean value
testdata['Age'] = testdata['Age'].fillna((testdata['Age'].mean()))

#Replacing 'NA' values in 'Year of Record' column with mean value
testdata['Year of Record']=testdata["Year of Record"].fillna(testdata['Year of Record'].mean())

#Replacing '0' values in 'University Degree' column with 'No'
testdata["University Degree"] = testdata["University Degree"].replace("0", "No")

#Replacing '0' values in 'Gender' column with 'unknown'
testdata["Gender"] = testdata["Gender"].replace("0", "unknown")


In [49]:
#taking feature columns from testdata
test_X = testdata[testdata.columns.difference(['Income'])]
test_y = testdata["Income"]

# Applying the model to the test data for prediction

In [50]:

incomePredicted=clf.predict(test_X)

In [51]:
#values predicted
incomePredicted

array([ 24870.52373185, -33720.92533708,  44416.38106865, ...,
        93143.37145569, 214444.03259029, 178854.61183767])

In [52]:
#replacing the values in testdata file with predicted values
testdata['Income']=incomePredicted


# Taking only instance and income column from the testdata file

In [53]:

finalfile=testdata[['Instance','Income']]

#Don't forget to add '.csv' at the end of the path
finalfile

,Instance,Income
0,111994,24870.523732
1,111995,-33720.925337
2,111996,44416.381069
3,111997,170909.974895
4,111998,12327.175693
...,...,...
73225,185219,115524.822661
73226,185220,110808.653727
73227,185221,93143.371456
73228,185222,214444.032590


# Exporting the final file

In [54]:

export_csv = finalfile.to_csv (r'tcd ml 2019-20 income prediction submission file1.csv', index = None, header=True)
